In [26]:
!pip install skl2onnx onnxmltools

In [27]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from google.colab import files
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from onnxmltools.convert import convert_xgboost
from onnxmltools.convert.common.data_types import FloatTensorType as FTT
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize




%matplotlib inline

https://www.kaggle.com/datasets/hassanamin/textdb3

1 is Fake
0 is Real

In [28]:
! gdown 1ZI9PMSKF1j2cJXZKNXc0EfzDJt3IwB-5

Downloading...
From: https://drive.google.com/uc?id=1ZI9PMSKF1j2cJXZKNXc0EfzDJt3IwB-5
To: /content/cleaned_final.csv
100% 28.8M/28.8M [00:00<00:00, 73.2MB/s]


In [29]:
df = pd.read_csv("cleaned_final.csv")
df.head()

,title,text,label
0,You Can Smell Hillary s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",1
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,1
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,0
3,Bernie supporters on Twitter erupt in anger ag...,"Kaydee King (@KaydeeKing) November 9, 2016 Th...",1
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,0


In [30]:
X = df["title"]+" "+df["text"]
y = df["label"]

In [31]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()
def lemma_tokenizer(text):
    tokens = word_tokenize(text.lower())
    return [lemmatizer.lemmatize(tok) for tok in tokens if tok.isalpha()]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

vectorizer = CountVectorizer(
    tokenizer=lemma_tokenizer,
    stop_words='english'
)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec  = vectorizer.transform(X_test)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'u', 'wa'] not in stop_words.
  warnings.warn(


#NB

In [32]:
model = MultinomialNB(alpha=0.5)
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9007936507936508
              precision    recall  f1-score   support

           0       0.90      0.91      0.91       659
           1       0.90      0.89      0.90       601

    accuracy                           0.90      1260
   macro avg       0.90      0.90      0.90      1260
weighted avg       0.90      0.90      0.90      1260



In [33]:
n_features = X_train_vec.shape[1]
initial_type = [('float_input', FloatTensorType([None, n_features]))]

target_opset = 12
options = { id(model): {"zipmap": False} }
onnx_model = convert_sklearn(
    model,
    initial_types=initial_type,
    options=options,
    target_opset=target_opset
)


onnx_model_path = "naive_bayes.onnx"
with open(onnx_model_path, "wb") as f:
    f.write(onnx_model.SerializeToString())

print(f"Model saved to {onnx_model_path}")

Model saved to naive_bayes.onnx


In [34]:
print(model.predict(vectorizer.transform(["Kicking the can down the road on tariffs won't work for this Maryland manufacturer Independent Can Company has raised prices twice this year already after Trump imposed 25% duties on steel in March, and then doubled them in June."])))
print(model.predict(vectorizer.transform(["Fact Sheet: The United States and European Union Reach Massive Trade Deal The White House July 28, 2025 MAKING GLOBAL HISTORY: Yesterday, announced a trade deal with the European Union (EU), fundamentally rebalancing the economic relationship between the world’s two largest economies. The deal marks a generational modernization of the transatlantic alliance and will provide Americans with unprecedented levels of market access to the European Union. The deal bolsters America’s economy and manufacturing capabilities. The EU will purchase $750 billion in U.S. energy and make new investments of $600 billion in the United States, all by 2028. Through decisive leadership and an unyielding commitment to American workers, President Trump has delivered yet another agreement that positions the United States as the world’s preeminent destination for investment, innovation, and advanced manufacturing. REACHING A HISTORIC TRADE DEAL: President Trump’s agreement with the European Union achieves historic structural reforms and strategic commitments that will benefit American industry, workers, and national security for generations: The United States and European Union reached a Cooperation Agreement on Reciprocal, Fair and Balanced Trade. This colossal deal will enable U.S. farmers, ranchers, fishermen, and manufacturers to increase U.S. exports, expand business opportunities, and help reduce the goods trade deficit with the European Union.  The EU will remove significant tariffs, including the elimination of all EU tariffs on U.S. industrial goods exported to the EU, creating enormous opportunities for American-made and American-grown goods to compete and win in Europe. This new market access will drive growth across the American economy—fueling exports, expanding production, and allowing American businesses of all sizes to reach millions of new customers across the Atlantic. As part of President Trump’s strategy to establish balanced trade, the European Union will pay the United States a tariff rate of 15%, including on autos and auto parts, pharmaceuticals, and semiconductors. However, the sectoral tariffs on steel, aluminum, and copper will remain unchanged—the EU will continue to pay 50% and the parties will discuss securing supply chains for these products. This new tariff regime will generate tens of billions of dollars in revenue annually and help to close the longstanding trade imbalance between the United States and Europe by encouraging local sourcing, reshoring production, and ensuring that foreign producers contribute their fair share to the American economy."])))


[0]
[1]


In [35]:
import numpy as np

feature_names = vectorizer.get_feature_names_out()

log_prob = model.feature_log_prob_

n_top = 20

for class_idx, class_label in enumerate(['REAL', 'FAKE']):
    print(f"\nTop {n_top} words for class '{class_label}':")

    top_feature_indices = np.argsort(log_prob[class_idx])[::-1][:n_top]

    for i in top_feature_indices:
        word = feature_names[i]
        prob = np.exp(log_prob[class_idx][i])
        print(f"{word}: {prob:.4f}")



Top 20 words for class 'REAL':
wa: 0.0113
said: 0.0108
trump: 0.0102
ha: 0.0096
clinton: 0.0070
state: 0.0067
republican: 0.0056
people: 0.0044
president: 0.0042
s: 0.0042
new: 0.0041
campaign: 0.0040
obama: 0.0037
say: 0.0036
year: 0.0034
time: 0.0033
party: 0.0033
candidate: 0.0031
american: 0.0030
like: 0.0027

Top 20 words for class 'FAKE':
wa: 0.0097
ha: 0.0083
clinton: 0.0066
trump: 0.0063
s: 0.0056
people: 0.0048
u: 0.0046
state: 0.0045
hillary: 0.0045
said: 0.0036
time: 0.0035
election: 0.0034
american: 0.0032
new: 0.0032
year: 0.0032
just: 0.0031
like: 0.0029
world: 0.0028
government: 0.0026
war: 0.0026


#SVM

In [36]:
svm = LinearSVC(max_iter=10000)
svm.fit(X_train_vec, y_train)

y_pred_svm = svm.predict(X_test_vec)

print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_svm))

SVM Accuracy: 0.9

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.90      0.90       659
           1       0.89      0.90      0.90       601

    accuracy                           0.90      1260
   macro avg       0.90      0.90      0.90      1260
weighted avg       0.90      0.90      0.90      1260



In [37]:
onnx_model = convert_sklearn(svm, initial_types=initial_type, target_opset=target_opset)
onnx_model_path = "svm.onnx"
with open(onnx_model_path, "wb") as f:
    f.write(onnx_model.SerializeToString())

print(f"Model saved to {onnx_model_path}")

Model saved to svm.onnx


In [38]:
print(svm.predict(vectorizer.transform(["Kicking the can down the road on tariffs won't work for this Maryland manufacturer Independent Can Company has raised prices twice this year already after Trump imposed 25% duties on steel in March, and then doubled them in June."])))
print(svm.predict(vectorizer.transform(["Fact Sheet: The United States and European Union Reach Massive Trade Deal The White House July 28, 2025 MAKING GLOBAL HISTORY: Yesterday, President Donald J. Trump announced a trade deal with the European Union (EU), fundamentally rebalancing the economic relationship between the world’s two largest economies. The deal marks a generational modernization of the transatlantic alliance and will provide Americans with unprecedented levels of market access to the European Union. The deal bolsters America’s economy and manufacturing capabilities. The EU will purchase $750 billion in U.S. energy and make new investments of $600 billion in the United States, all by 2028. Through decisive leadership and an unyielding commitment to American workers, President Trump has delivered yet another agreement that positions the United States as the world’s preeminent destination for investment, innovation, and advanced manufacturing. REACHING A HISTORIC TRADE DEAL: President Trump’s agreement with the European Union achieves historic structural reforms and strategic commitments that will benefit American industry, workers, and national security for generations: The United States and European Union reached a Cooperation Agreement on Reciprocal, Fair and Balanced Trade. This colossal deal will enable U.S. farmers, ranchers, fishermen, and manufacturers to increase U.S. exports, expand business opportunities, and help reduce the goods trade deficit with the European Union.  The EU will remove significant tariffs, including the elimination of all EU tariffs on U.S. industrial goods exported to the EU, creating enormous opportunities for American-made and American-grown goods to compete and win in Europe. This new market access will drive growth across the American economy—fueling exports, expanding production, and allowing American businesses of all sizes to reach millions of new customers across the Atlantic. As part of President Trump’s strategy to establish balanced trade, the European Union will pay the United States a tariff rate of 15%, including on autos and auto parts, pharmaceuticals, and semiconductors. However, the sectoral tariffs on steel, aluminum, and copper will remain unchanged—the EU will continue to pay 50% and the parties will discuss securing supply chains for these products. This new tariff regime will generate tens of billions of dollars in revenue annually and help to close the longstanding trade imbalance between the United States and Europe by encouraging local sourcing, reshoring production, and ensuring that foreign producers contribute their fair share to the American economy."])))


[0]
[0]


In [39]:
import numpy as np

feature_names = vectorizer.get_feature_names_out()


coefs = svm.coef_[0]

n_top = 20

top_fake_indices = np.argsort(coefs)[::-1][:n_top]
print(f"\nTop {n_top} words for class 'FAKE':")
for i in top_fake_indices:
    print(f"{feature_names[i]}: {coefs[i]:.4f}")

top_real_indices = np.argsort(coefs)[:n_top]
print(f"\nTop {n_top} words for class 'REAL':")
for i in top_real_indices:
    print(f"{feature_names[i]}: {coefs[i]:.4f}")



Top 20 words for class 'FAKE':
october: 0.3909
cave: 0.3224
dog: 0.2984
flashback: 0.2966
share: 0.2941
remixed: 0.2906
controlled: 0.2837
conservatism: 0.2629
dad: 0.2618
mormon: 0.2602
rise: 0.2514
populism: 0.2451
print: 0.2430
reader: 0.2287
prepare: 0.2147
source: 0.2129
establishment: 0.2113
ibid: 0.2102
zika: 0.2102
identity: 0.2087

Top 20 words for class 'REAL':
feud: -0.4817
quote: -0.4753
nailing: -0.3532
gop: -0.3337
passion: -0.3151
easier: -0.3017
trade: -0.3017
insist: -0.2910
consequence: -0.2821
rush: -0.2747
protective: -0.2690
bernie: -0.2669
defending: -0.2665
shortage: -0.2587
demo: -0.2538
sweeping: -0.2537
naked: -0.2483
fit: -0.2466
failed: -0.2427
transition: -0.2399


#XGBoost

In [40]:
xgb = XGBClassifier(eval_metric='logloss')
xgb.fit(X_train_vec, y_train)

y_pred_xgb = xgb.predict(X_test_vec)
print("XGB Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_xgb))

XGB Accuracy: 0.9253968253968254

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.91      0.93       659
           1       0.91      0.94      0.92       601

    accuracy                           0.93      1260
   macro avg       0.93      0.93      0.93      1260
weighted avg       0.93      0.93      0.93      1260



In [41]:
n_features = len(vectorizer.vocabulary_)
initial_types = [
    ('input', FTT([None, n_features]))
]

onnx_xgb = convert_xgboost(
    xgb,
    initial_types=initial_types,
    target_opset=target_opset
)

with open("xgboost.onnx", "wb") as f:
    f.write(onnx_xgb.SerializeToString())

In [42]:
print(xgb.predict(vectorizer.transform(["Kicking the can down the road on tariffs won't work for this Maryland manufacturer Independent Can Company has raised prices twice this year already after Trump imposed 25% duties on steel in March, and then doubled them in June."])))
print(xgb.predict(vectorizer.transform(["Fact Sheet: The United States and European Union Reach Massive Trade Deal The White House July 28, 2025 MAKING GLOBAL HISTORY: Yesterday, President Donald J. Trump announced a trade deal with the European Union (EU), fundamentally rebalancing the economic relationship between the world’s two largest economies. The deal marks a generational modernization of the transatlantic alliance and will provide Americans with unprecedented levels of market access to the European Union. The deal bolsters America’s economy and manufacturing capabilities. The EU will purchase $750 billion in U.S. energy and make new investments of $600 billion in the United States, all by 2028. Through decisive leadership and an unyielding commitment to American workers, President Trump has delivered yet another agreement that positions the United States as the world’s preeminent destination for investment, innovation, and advanced manufacturing. REACHING A HISTORIC TRADE DEAL: President Trump’s agreement with the European Union achieves historic structural reforms and strategic commitments that will benefit American industry, workers, and national security for generations: The United States and European Union reached a Cooperation Agreement on Reciprocal, Fair and Balanced Trade. This colossal deal will enable U.S. farmers, ranchers, fishermen, and manufacturers to increase U.S. exports, expand business opportunities, and help reduce the goods trade deficit with the European Union.  The EU will remove significant tariffs, including the elimination of all EU tariffs on U.S. industrial goods exported to the EU, creating enormous opportunities for American-made and American-grown goods to compete and win in Europe. This new market access will drive growth across the American economy—fueling exports, expanding production, and allowing American businesses of all sizes to reach millions of new customers across the Atlantic. As part of President Trump’s strategy to establish balanced trade, the European Union will pay the United States a tariff rate of 15%, including on autos and auto parts, pharmaceuticals, and semiconductors. However, the sectoral tariffs on steel, aluminum, and copper will remain unchanged—the EU will continue to pay 50% and the parties will discuss securing supply chains for these products. This new tariff regime will generate tens of billions of dollars in revenue annually and help to close the longstanding trade imbalance between the United States and Europe by encouraging local sourcing, reshoring production, and ensuring that foreign producers contribute their fair share to the American economy."])))


[0]
[1]


In [43]:
feature_names = vectorizer.get_feature_names_out()

importances = xgb.feature_importances_

n_top = 20
top_indices = np.argsort(importances)[::-1][:n_top]

print(f"\nTop {n_top} most important words (XGBoost):")
for i in top_indices:
    print(f"{feature_names[i]}: {importances[i]:.4f}")


Top 20 most important words (XGBoost):
gop: 0.0254
republican: 0.0219
cruz: 0.0134
said: 0.0133
nomination: 0.0110
october: 0.0107
reform: 0.0086
fbi: 0.0084
stolen: 0.0081
http: 0.0080
obama: 0.0078
rubio: 0.0076
article: 0.0074
november: 0.0074
obamacare: 0.0071
percent: 0.0070
hatred: 0.0068
jewish: 0.0067
debate: 0.0061
print: 0.0061


In [44]:
1/0

#Exporting

In [45]:
files.download("naive_bayes.onnx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [46]:
files.download("svm.onnx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [47]:
files.download("xgboost.onnx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [48]:
import json

with open("vectorizer.json", "w") as f:
    json.dump(vectorizer.vocabulary_, f)

files.download("vectorizer.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>